## Configure DSPy environment

In [6]:
# import dspy
import dspy

# For display
from IPython.display import Markdown

In [3]:
# Lets tell dspy that we will use OpenAI GPT 4o mini
lm = dspy.LM('openai/gpt-4o-mini')
dspy.configure(lm = lm)

## Exploring some DSPy modules

In [7]:
qa = dspy.Predict('question : str -> response : str')
Markdown(qa(question = 'What are high memory and low memory in linux').response)

In Linux, "high memory" and "low memory" refer to different regions of the system's memory address space, particularly in the context of 32-bit architectures.

- **Low Memory**: This typically refers to the first 896 MB of memory in a 32-bit system. It is directly accessible by the kernel and can be used for kernel data structures and user processes. The low memory region is where most of the system's memory management occurs, and it is where the kernel can allocate memory for processes without needing special handling.

- **High Memory**: This refers to the memory above the 896 MB threshold in a 32-bit system. This memory is not directly accessible by the kernel in the same way as low memory. Instead, it requires special handling, such as using a mechanism called "high memory management" to access it. High memory is typically used for user processes and can be allocated dynamically, but the kernel must map it into its address space to access it.

In summary, low memory is directly accessible by the kernel, while high memory requires additional management to be accessed by the kernel in a 32-bit Linux environment.

## Inspection

In [10]:
dspy.inspect_history(n=1)





[2024-10-19T13:05:29.277664]

System message:

Your input fields are:
1. `question` (str)

Your output fields are:
1. `response` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `response`.


User message:

[[ ## question ## ]]
What are high memory and low memory in linux

Respond with the corresponding output fields, starting with the field `response`, and then ending with the marker for `completed`.


Response:

[[ ## response ## ]]
In Linux, "high memory" and "low memory" refer to different regions of the system's memory address space, particularly in the context of 32-bit architectures.

- **Low Memory**: This typically refers to the first 896 MB of memory in a 32-bit system. It is directly accessible by the kernel and can be used

In [16]:
cot = dspy.ChainOfThought('question -> response')
cot(question = "should curly braces appear on their own line ?")

Prediction(
    reasoning='The placement of curly braces often depends on the coding style guidelines being followed. In many programming languages, such as Java, C#, and JavaScript, it is common to place opening curly braces on the same line as the statement that precedes them, while closing curly braces are often placed on their own line. This style enhances readability and maintains a clear structure. However, some coding standards, like those used in Python or certain functional programming languages, may not use curly braces at all. Ultimately, whether curly braces should appear on their own line is a matter of personal or team preference, and consistency within a codebase is key.',
    response="Curly braces can appear on their own line depending on the coding style guidelines you are following. Many developers prefer placing the opening brace on the same line as the preceding statement for readability, while the closing brace is often placed on its own line. It's important to fo

In [17]:
dspy.inspect_history(n=1)





[2024-10-19T13:14:49.784757]

System message:

Your input fields are:
1. `question` (str)

Your output fields are:
1. `reasoning` (str)
2. `response` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `response`.


User message:

[[ ## question ## ]]
should curly braces appear on their own line ?

Respond with the corresponding output fields, starting with the field `reasoning`, then `response`, and then ending with the marker for `completed`.


Response:

[[ ## reasoning ## ]]
The placement of curly braces often depends on the coding style guidelines being followed. In many programming languages, such as Java, C#, and JavaScript, it is common to place opening curly braces on the same line as the state

## Manipulating Examples ins DSPy

In [19]:
import ujson

# Download 500 question--answer pairs from the RAG-QA Arena "Tech" dataset.
# !wget https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_500.json
# CURL -O https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_500.json

with open('data/ragqa_arena_tech_500.json') as f:
    data = ujson.load(f)

# Inspect one datapoint.
data[0]


{'question': 'how to transfer whatsapp voice message to computer?',
 'response': 'To transfer voice notes from WhatsApp on your device to your computer, you have the option to select the "Share" feature within the app and send the files via Email, Gmail, Bluetooth, or other available services.  \nYou can also move the files onto your phone\'s SD card, connect your phone to your computer via a USB cable, then find and transfer the files via File Explorer on your PC. \nAlternatively, you can choose to attach all the desired voice notes to an email and, from your phone, send them to your own email address.  \nUpon receiving the email on your computer, you can then download the voice note attachments.'}